In [1]:
import pandas as pd
from glob import glob
from tqdm.notebook import tqdm
from scipy.stats import ttest_ind, mannwhitneyu, shapiro
import requests
from bs4 import BeautifulSoup
import sys

In [2]:
from rpy2 import robjects
from rpy2.robjects import Formula

from rpy2.robjects import pandas2ri
pandas2ri.activate()
from rpy2.robjects.packages import importr

base = importr("base")
stats = importr("stats")
DESeq2 = importr("DESeq2")

During startup - Warning messages:
1: Setting LC_COLLATE failed, using "C" 
2: Setting LC_TIME failed, using "C" 
3: Setting LC_MESSAGES failed, using "C" 
4: Setting LC_MONETARY failed, using "C" 


In [3]:
upar = pd.read_csv('human_upar.tsv', sep='\t')['Approved symbol']
upar = sorted(map(lambda x: x.lower(), set(upar)))
len(upar)

35

In [4]:
path = 'results/mouse/'

In [5]:
def deseq(meta, counts, formula, ref, neg):
    meta["Tissue"] = stats.relevel(robjects.vectors.FactorVector(meta["Tissue"]), ref=ref)

    # Calculate normalization factors
    dds = DESeq2.DESeqDataSetFromMatrix(countData=counts, colData=meta, design=Formula(formula))
    dds = DESeq2.DESeq(dds)

    res = DESeq2.results(dds, name=f"Tissue_{neg}_vs_{ref}")
    res = DESeq2.lfcShrink(dds, coef=f"Tissue_{neg}_vs_{ref}", type="apeglm")
    res = pd.DataFrame(base.as_data_frame(res))
    res.index = counts.index
    res = res.sort_values("padj")
    res = res.loc[res["padj"] < 0.05]
#     res = res.loc[res["log2FoldChange"].abs() > 0.5]

    return res

In [6]:
def desec_for_df(tmp, ad, wt):
    ads = tmp[ad].shape[1]
    wts = tmp[wt].shape[1]
    tmp = tmp[[ad, wt]].copy()
    tmp.columns = tmp.columns + "_" + pd.Series(list(range(1, ads + 1)) + list(range(1, wts + 1))).astype(str)

    meta = pd.DataFrame({"Tissue": [ad] * ads + [wt] * wts}, index=tmp.columns)
    des = deseq(meta=meta, counts=tmp, formula="~ Tissue", ref=wt, neg=ad)
    return des

# GSE178662

In [7]:
with open('GSE178662/test_html') as f:
    soup = BeautifulSoup(f.read(), 'html.parser')

In [8]:
html = {}
tables = soup.findAll('table', {'cellpadding': '2', 'cellspacing': '0', 'width': 600})
for tab in tables:
    my_list = tab.findAll('td', {"style":"text-align: justify"})
    tmp = my_list[2].text.rsplit(' oldsite')[0].split(': ')
    if tmp[1] == '3xTG-ADage':
        tmp[1] = 'AD'
    else:
        tmp[1] = 'WT'
    html[my_list[0].text] = tmp[1] + '_' + tmp[2].split()[0]

In [9]:
df = pd.read_csv('GSE178662/GSE178662_Gene_counts_filtered_GEO.txt', sep='\t')
df.index = df.index.str.split('.').str[0]
df.columns = df.columns.map(html.get)
df

,AD_6,AD_6,AD_6,AD_6,AD_6,AD_6,AD_6,AD_6,WT_6,WT_6,...,AD_6,AD_6,AD_6,AD_6,WT_3,WT_3,WT_3,AD_3,AD_3,AD_3
ENSMUSG00000098104,52,48,68,71,56,77,36,41,78,23,...,5,8,12,52,23,33,95,53,33,41
ENSMUSG00000033845,261,166,289,173,245,245,70,110,149,146,...,64,24,97,132,70,113,210,166,131,108
ENSMUSG00000025903,380,385,823,400,684,712,250,318,593,406,...,119,62,172,250,339,245,1856,559,333,370
ENSMUSG00000033813,232,197,386,128,258,282,89,234,203,182,...,240,332,145,200,126,99,129,140,147,92
ENSMUSG00000002459,6,3,9,2,5,14,1,5,1,2,...,4,4,1,11,4,1,1,4,3,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSMUSG00000064368,6080,4522,9553,1156,9078,5204,2020,9297,16505,14937,...,925,1358,1313,5074,10010,4462,9070,7227,5730,7415
ENSMUSG00000064369,1664,1036,1418,554,1924,1386,350,1244,3274,2340,...,130,87,145,387,1110,1063,3673,1272,631,943
ENSMUSG00000064370,45652,42456,57377,16426,115944,76808,26250,87957,148005,185414,...,28365,34442,26160,58496,90577,39119,166211,69565,48249,65051
ENSMUSG00000064371,90,51,166,145,16,275,101,105,26,71,...,13,16,5,14,210,354,0,151,233,40


In [10]:
def get_uniprot(name):
    requestURL = f"https://www.ebi.ac.uk/proteins/api/proteins?offset=0&size=100&gene={name.lower()}&organism=Mus%20musculus"

    r = requests.get(requestURL, headers={ "Accept" : "application/json"})

    if not r.ok:
        r.raise_for_status()
        sys.exit()

    return r.json()

def get_symbols(name):
    json = get_uniprot(name)
    res = set()
    if isinstance(json, list):
        for d in json:
            for i in d['dbReferences']:
                if i['type'] == 'VEuPathDB':
                    res.add(i['id'].split(':')[1])
    return list(res)

In [11]:
res = {}
for gene in tqdm(upar):
    print(gene, end=', ')
    res[gene] = get_symbols(gene)
res = {i: k for k, v in res.items() for i in v }

  0%|          | 0/35 [00:00<?, ?it/s]

cd177, cd59, gml, gpihbp1, ly6d, ly6e, ly6g5b, ly6g5c, ly6g6c, ly6g6d, ly6g6e, ly6g6f, ly6h, ly6k, ly6l, lynx1, lypd1, lypd2, lypd3, lypd4, lypd5, lypd6, lypd6b, lypd8, pate1, pate2, pate3, pate4, pinlyp, plaur, psca, slurp1, slurp2, spaca4, tex101, 

In [12]:
df = df.reindex(res).dropna()
df.index = df.index.map(res.get)
df

,AD_6,AD_6,AD_6,AD_6,AD_6,AD_6,AD_6,AD_6,WT_6,WT_6,...,AD_6,AD_6,AD_6,AD_6,WT_3,WT_3,WT_3,AD_3,AD_3,AD_3
cd177,18.0,47.0,85.0,46.0,270.0,96.0,127.0,86.0,21.0,19.0,...,16.0,30.0,108.0,120.0,18.0,36.0,0.0,100.0,48.0,67.0
cd59,133.0,135.0,87.0,44.0,106.0,143.0,70.0,289.0,223.0,347.0,...,171.0,107.0,128.0,200.0,203.0,132.0,59.0,161.0,153.0,124.0
cd59,2222.0,1735.0,3499.0,1454.0,3182.0,2493.0,1607.0,4260.0,4474.0,6328.0,...,2006.0,3145.0,3396.0,2841.0,1318.0,2254.0,1495.0,1699.0,1875.0,941.0
ly6d,2221.0,1868.0,2824.0,1303.0,3038.0,4491.0,586.0,1892.0,3888.0,4279.0,...,470.0,59.0,255.0,1892.0,1491.0,1003.0,3969.0,2021.0,1502.0,1351.0
ly6e,4328.0,4489.0,12964.0,6300.0,10577.0,9375.0,4291.0,2562.0,6957.0,3525.0,...,1662.0,442.0,1161.0,4826.0,3194.0,3912.0,11896.0,10849.0,6445.0,6596.0
ly6g5b,0.0,8.0,37.0,6.0,15.0,10.0,12.0,28.0,18.0,3.0,...,13.0,0.0,10.0,13.0,0.0,5.0,19.0,13.0,22.0,3.0
ly6g6c,208.0,186.0,681.0,215.0,418.0,894.0,207.0,281.0,1085.0,1080.0,...,160.0,157.0,323.0,404.0,479.0,725.0,1997.0,248.0,403.0,159.0
ly6g6d,26.0,23.0,81.0,34.0,133.0,102.0,33.0,99.0,131.0,113.0,...,48.0,17.0,57.0,61.0,32.0,36.0,146.0,71.0,65.0,26.0
ly6g6e,0.0,3.0,6.0,0.0,15.0,1.0,0.0,0.0,3.0,0.0,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,14.0,11.0
ly6g6f,176.0,102.0,489.0,217.0,252.0,368.0,144.0,174.0,690.0,954.0,...,61.0,173.0,285.0,404.0,628.0,706.0,821.0,73.0,157.0,119.0


In [13]:
df.columns.value_counts()

AD_6    23
WT_6    13
WT_3     3
AD_3     3
dtype: int64

In [14]:
result = pd.DataFrame()

In [15]:
d = desec_for_df(df, 'AD_3', 'WT_3')
result = pd.concat([result, d])
d.to_csv(path + 'GSE178662_3.tsv', sep='\t')
d

R[write to console]: converting counts to integer mode

R[write to console]: estimating size factors

R[write to console]: estimating dispersions

R[write to console]: gene-wise dispersion estimates

R[write to console]: mean-dispersion relationship

R[write to console]: final dispersion estimates

R[write to console]: fitting model and testing

R[write to console]: using 'apeglm' for LFC shrinkage. If used in published research, please cite:
    Zhu, A., Ibrahim, J.G., Love, M.I. (2018) Heavy-tailed prior distributions for
    sequence count data: removing the noise and preserving large differences.
    Bioinformatics. https://doi.org/10.1093/bioinformatics/bty895



,baseMean,log2FoldChange,lfcSE,pvalue,padj
ly6g6f,411.666576,-2.283705,0.562713,0.000004,0.000079
ly6g6c,547.785624,-1.434180,0.424190,0.000102,0.001127
ly6e,6410.495560,0.641378,0.226321,0.002400,0.017599


In [16]:
d = desec_for_df(df, 'AD_6', 'WT_6')
result = pd.concat([result, d])
d.to_csv(path + 'GSE178662_6.tsv', sep='\t')
d

R[write to console]: converting counts to integer mode

R[write to console]: estimating size factors

R[write to console]: estimating dispersions

R[write to console]: gene-wise dispersion estimates

R[write to console]: mean-dispersion relationship

R[write to console]: final dispersion estimates

R[write to console]: fitting model and testing

R[write to console]: -- replacing outliers and refitting for 5 genes
-- DESeq argument 'minReplicatesForReplace' = 7 
-- original counts are preserved in counts(dds)

R[write to console]: estimating dispersions

R[write to console]: fitting model and testing

R[write to console]: using 'apeglm' for LFC shrinkage. If used in published research, please cite:
    Zhu, A., Ibrahim, J.G., Love, M.I. (2018) Heavy-tailed prior distributions for
    sequence count data: removing the noise and preserving large differences.
    Bioinformatics. https://doi.org/10.1093/bioinformatics/bty895



,baseMean,log2FoldChange,lfcSE,pvalue,padj
lypd5,60.104491,2.396339,0.359297,2.202576e-12,4.845667e-11
cd177,104.969293,2.141955,0.415869,2.342494e-08,2.576743e-07
ly6g6c,500.592596,-0.800141,0.165355,7.822016e-07,5.736145e-06
plaur,220.061613,1.314751,0.287823,1.207491e-06,6.641201e-06
ly6g5b,9.180701,2.042862,0.596032,6.001914e-05,2.640842e-04
lypd6,2.770418,-1.720919,0.744186,2.978128e-03,1.091980e-02
ly6d,1683.234672,-0.768637,0.301815,6.322915e-03,1.987202e-02
ly6k,28.977118,1.098759,0.553035,1.259146e-02,3.462651e-02
ly6g6f,340.403486,-0.506444,0.220972,1.726161e-02,4.219504e-02


In [17]:
result = result.sort_values('padj').reset_index().drop_duplicates('index', keep='first').set_index('index')
result.to_csv(path + 'GSE178662.tsv', sep='\t')
result.to_excel(path + 'GSE178662.xlsx')
result

,baseMean,log2FoldChange,lfcSE,pvalue,padj
index,,,,,
lypd5,60.104491,2.396339,0.359297,2.202576e-12,4.845667e-11
cd177,104.969293,2.141955,0.415869,2.342494e-08,2.576743e-07
ly6g6c,500.592596,-0.800141,0.165355,7.822016e-07,5.736145e-06
plaur,220.061613,1.314751,0.287823,1.207491e-06,6.641201e-06
ly6g6f,411.666576,-2.283705,0.562713,3.568962e-06,7.851716e-05
ly6g5b,9.180701,2.042862,0.596032,6.001914e-05,2.640842e-04
lypd6,2.770418,-1.720919,0.744186,2.978128e-03,1.091980e-02
ly6e,6410.495560,0.641378,0.226321,2.399861e-03,1.759898e-02
ly6d,1683.234672,-0.768637,0.301815,6.322915e-03,1.987202e-02


In [18]:
pd.DataFrame(result.padj)

,padj
index,
lypd5,4.845667e-11
cd177,2.576743e-07
ly6g6c,5.736145e-06
plaur,6.641201e-06
ly6g6f,7.851716e-05
ly6g5b,2.640842e-04
lypd6,1.091980e-02
ly6e,1.759898e-02
ly6d,1.987202e-02


# GSE164798

In [19]:
result = pd.DataFrame()

In [20]:
df = pd.read_csv('GSE164798/GSE164798_Raw_gene_counts_matrix.txt', sep='\t', index_col=0).iloc[:, 5:]
df.columns = df.columns.str.rsplit('.', 5).str[0].str.split('_', 5).str[-1].str.rsplit('_', 3).str[0]
df.index = df.index.str.lower()
df = df.reindex(upar).dropna()
df

,wt_A,wt_A,wt_A,wt_A,wt_B,wt_B,wt_B,wt_B,wt_C,wt_C,...,tg_A,tg_A,tg_B,tg_B,tg_B,tg_B,tg_C,tg_C,tg_C,tg_C
Geneid,,,,,,,,,,,,,,,,,,,,,
cd177,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,...,1.0,0.0,2.0,0.0,2.0,2.0,0.0,2.0,2.0,2.0
gml,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
gpihbp1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,1.0,1.0
ly6d,5.0,1.0,9.0,5.0,2.0,7.0,3.0,12.0,3.0,2.0,...,2.0,3.0,5.0,3.0,7.0,7.0,1.0,7.0,1.0,2.0
ly6e,3032.0,2834.0,2931.0,3202.0,2671.0,4155.0,2832.0,3340.0,3134.0,2566.0,...,3565.0,4241.0,4593.0,2352.0,3151.0,2688.0,3585.0,3377.0,2562.0,2094.0
ly6g5b,2.0,2.0,0.0,4.0,5.0,0.0,1.0,1.0,1.0,7.0,...,2.0,1.0,1.0,3.0,1.0,2.0,5.0,2.0,4.0,2.0
ly6g5c,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
ly6g6c,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
ly6g6d,16.0,8.0,7.0,7.0,21.0,6.0,2.0,11.0,4.0,12.0,...,11.0,8.0,16.0,8.0,14.0,2.0,14.0,7.0,8.0,7.0


In [21]:
df.columns.value_counts()

wt_A    4
wt_B    4
wt_C    4
tg_A    4
tg_B    4
tg_C    4
dtype: int64

In [22]:
d = desec_for_df(df, 'tg_A', 'wt_A')
result = pd.concat([result, d])
d.to_csv(path + 'GSE164798_A.tsv', sep='\t')
d

R[write to console]: converting counts to integer mode

R[write to console]: estimating size factors

R[write to console]: estimating dispersions

R[write to console]: gene-wise dispersion estimates

R[write to console]: mean-dispersion relationship

R[write to console]: final dispersion estimates

R[write to console]: fitting model and testing

R[write to console]: using 'apeglm' for LFC shrinkage. If used in published research, please cite:
    Zhu, A., Ibrahim, J.G., Love, M.I. (2018) Heavy-tailed prior distributions for
    sequence count data: removing the noise and preserving large differences.
    Bioinformatics. https://doi.org/10.1093/bioinformatics/bty895



,baseMean,log2FoldChange,lfcSE,pvalue,padj
Geneid,,,,,
ly6h,2331.13022,-0.408488,0.172067,0.001037,0.023848


In [23]:
d = desec_for_df(df, 'tg_B', 'wt_B')
result = pd.concat([result, d])
d.to_csv(path + 'GSE164798_B.tsv', sep='\t')
d

R[write to console]: converting counts to integer mode

R[write to console]: estimating size factors

R[write to console]: estimating dispersions

R[write to console]: gene-wise dispersion estimates

R[write to console]: mean-dispersion relationship

R[write to console]: final dispersion estimates

R[write to console]: fitting model and testing

R[write to console]: using 'apeglm' for LFC shrinkage. If used in published research, please cite:
    Zhu, A., Ibrahim, J.G., Love, M.I. (2018) Heavy-tailed prior distributions for
    sequence count data: removing the noise and preserving large differences.
    Bioinformatics. https://doi.org/10.1093/bioinformatics/bty895



,baseMean,log2FoldChange,lfcSE,pvalue,padj
Geneid,,,,,
ly6h,2484.917151,-0.674969,0.16765,0.000004,0.000085


In [24]:
d = desec_for_df(df, 'tg_C', 'wt_C')
result = pd.concat([result, d])
d.to_csv(path + 'GSE164798_C.tsv', sep='\t')
d

R[write to console]: converting counts to integer mode

R[write to console]: estimating size factors

R[write to console]: estimating dispersions

R[write to console]: gene-wise dispersion estimates

R[write to console]: mean-dispersion relationship

R[write to console]: final dispersion estimates

R[write to console]: fitting model and testing

R[write to console]: using 'apeglm' for LFC shrinkage. If used in published research, please cite:
    Zhu, A., Ibrahim, J.G., Love, M.I. (2018) Heavy-tailed prior distributions for
    sequence count data: removing the noise and preserving large differences.
    Bioinformatics. https://doi.org/10.1093/bioinformatics/bty895



,baseMean,log2FoldChange,lfcSE,pvalue,padj
Geneid,,,,,


In [25]:
result = result.sort_values('padj').reset_index().drop_duplicates('Geneid', keep='first').set_index('Geneid')
result.to_csv(path + 'GSE164798.tsv', sep='\t')
result.to_excel(path + 'GSE164798.xlsx')
result

,baseMean,log2FoldChange,lfcSE,pvalue,padj
Geneid,,,,,
ly6h,2484.917151,-0.674969,0.16765,0.000004,0.000085


In [26]:
pd.DataFrame(result.padj)

,padj
Geneid,
ly6h,0.000085


# GSE169686

In [27]:
result = pd.DataFrame()

In [28]:
df = pd.read_csv('GSE169686/GSE169686_3mo_counts.txt', sep='\t')
df = df.sort_values(df.columns[2:].tolist(), ascending=False).drop_duplicates('NAME', keep='first')
df = df.set_index('NAME').drop('DESCRIPTION', axis=1)
df.index = df.index.str.lower()
df = df.reindex(upar).dropna()
df.columns = df.columns.str.split('.', 1).str[1]
df

,1mpi.minus,1mpi.minus,1mpi.minus,1mpi.minus,1mpi.minus,1mpi.plus,1mpi.plus,1mpi.plus,1mpi.plus,1mpi.plus,...,3mpi.plus,3mpi.plus,5mpi.minus,5mpi.minus,5mpi.minus,5mpi.minus,5mpi.minus,5mpi.plus,5mpi.plus,5mpi.plus
NAME,,,,,,,,,,,,,,,,,,,,,
cd177,0.086166,0.158338,0.000000,0.040148,0.164471,0.075804,0.064075,0.133220,0.020224,0.081354,...,0.109297,0.214599,0.067405,0.122504,0.176908,0.124955,0.063547,0.112629,0.407088,0.041191
gml,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
gpihbp1,0.064624,0.090479,0.066207,0.000000,0.164471,0.037902,0.042716,0.000000,0.182017,0.000000,...,0.131156,0.250366,0.022468,0.040835,0.039313,0.000000,0.000000,0.022526,0.000000,0.000000
ly6d,0.366204,0.113098,0.066207,0.160593,0.267266,0.094755,0.000000,0.044407,0.546052,0.101692,...,0.131156,0.214599,0.022468,0.081669,0.039313,0.229084,0.169460,0.067578,0.116311,0.020595
ly6e,144.650481,171.660757,142.610712,161.797488,157.604414,170.256041,168.281475,138.504638,197.630243,158.314881,...,146.085964,149.647308,145.639608,160.459745,173.232663,165.731667,170.370418,153.356023,196.196866,169.190521
ly6g5b,0.021541,0.022620,0.132415,0.080297,0.082236,0.132657,0.000000,0.066610,0.000000,0.081354,...,0.109297,0.125183,0.044937,0.102087,0.117939,0.166606,0.105912,0.045052,0.174466,0.020595
ly6g5c,0.000000,0.000000,0.000000,0.000000,0.000000,0.018951,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ly6g6c,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ly6g6d,0.258497,0.361915,0.110346,0.341260,0.370060,0.341119,0.128149,0.222034,0.465155,0.305077,...,0.437187,0.304016,0.224683,0.163339,0.196565,0.249910,0.148277,0.112629,0.058155,0.226548


In [29]:
df.columns.value_counts()

1mpi.plus     6
1mpi.minus    5
3mpi.minus    5
3mpi.plus     5
5mpi.minus    5
5mpi.plus     3
dtype: int64

In [30]:
def mw_count(df, wt, ad, pval=0.06):
    mw = pd.DataFrame([mannwhitneyu(df.loc[gene, wt], df.loc[gene, ad]) for gene in df.index],
                      index=df.index, columns=["mw_pcx", 'p-value_pcx'])
    pcx = mw[mw['p-value_pcx'] < pval].sort_values('p-value_pcx')
    return pcx

In [31]:
d = mw_count(df, '1mpi.plus', '1mpi.minus')
result = pd.concat([result, d])
d

,mw_pcx,p-value_pcx
NAME,,
ly6k,3.0,0.030303


In [32]:
d = mw_count(df, '3mpi.plus', '3mpi.minus')
result = pd.concat([result, d])
d

,mw_pcx,p-value_pcx
NAME,,


In [33]:
d = mw_count(df, '5mpi.plus', '5mpi.minus')
result = pd.concat([result, d])
d

,mw_pcx,p-value_pcx
NAME,,
lynx1,15.0,0.035714


In [34]:
df = pd.read_csv('GSE169686/GSE169686_12mo_counts.txt', sep='\t')
df = df.sort_values(df.columns[2:].tolist(), ascending=False).drop_duplicates('NAME', keep='first')
df = df.set_index('NAME').drop('DESCRIPTION', axis=1)
df.index = df.index.str.lower()
df = df.reindex(upar).dropna()
df.columns = df.columns.str.split('.', 1).str[1]
df

,1mpi.neg,1mpi.neg,1mpi.neg,1mpi.neg,1mpi.neg,1mpi.pos,1mpi.pos,1mpi.pos,1mpi.pos,1mpi.pos,...,5mpi.neg,5mpi.neg,5mpi.neg,5mpi.pos,5mpi.pos,5mpi.pos,5mpi.pos,5mpi.pos,5mpi.pos,5mpi.pos
NAME,,,,,,,,,,,,,,,,,,,,,
cd177,0.000000,0.328326,0.268002,0.038718,0.193132,0.092699,0.088181,0.200262,0.071743,0.150215,...,0.021954,0.065441,0.147242,0.180498,0.110966,0.068423,0.124195,0.084427,0.200032,0.203001
gml,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
gpihbp1,0.000000,0.000000,0.111668,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.042919,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.062098,0.000000,0.044451,0.000000
ly6d,0.071075,0.231759,0.134001,0.038718,0.444203,0.278098,0.000000,0.120157,0.556011,0.128756,...,0.065861,0.043628,0.126208,0.100277,0.266319,0.051317,0.124195,0.675419,0.244483,0.496225
ly6e,146.289557,163.583418,164.330121,166.468987,170.844292,159.609731,160.599838,166.037325,156.131575,180.494604,...,175.716588,162.861625,167.288444,157.073548,161.345023,146.664119,162.074851,149.162015,137.644001,178.415605
ly6g5b,0.071075,0.270386,0.178668,0.000000,0.231758,0.037080,0.154317,0.040052,0.125551,0.107297,...,0.000000,0.000000,0.042069,0.020055,0.066580,0.119740,0.041398,0.042214,0.000000,0.000000
ly6g5c,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ly6g6c,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.071743,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ly6g6d,0.213224,0.328326,0.089334,0.096796,0.405577,0.296638,0.440906,0.300393,0.197294,0.193134,...,0.351258,0.239951,0.462762,0.240664,0.288512,0.444748,0.579578,0.422137,0.177806,0.112779


In [35]:
df.columns.value_counts()

5mpi.neg    7
5mpi.pos    7
1mpi.pos    6
3mpi.neg    6
3mpi.pos    6
1mpi.neg    5
dtype: int64

In [36]:
d = mw_count(df, '1mpi.neg', '1mpi.pos')
result = pd.concat([result, d])
d

,mw_pcx,p-value_pcx
NAME,,
lypd8,29.0,0.012866
pate2,3.0,0.030303


In [37]:
d = mw_count(df, '3mpi.neg', '3mpi.pos')
result = pd.concat([result, d])
d

,mw_pcx,p-value_pcx
NAME,,


In [38]:
d = mw_count(df, '5mpi.neg', '5mpi.pos')
result = pd.concat([result, d])
d

,mw_pcx,p-value_pcx
NAME,,


In [39]:
result = result.sort_values('p-value_pcx').reset_index().drop_duplicates('NAME', keep='first').set_index('NAME')
result.to_csv(path + 'GSE169686.tsv', sep='\t')
result.to_excel(path + 'GSE169686.xlsx')
result

,mw_pcx,p-value_pcx
NAME,,
lypd8,29.0,0.012866
ly6k,3.0,0.030303
pate2,3.0,0.030303
lynx1,15.0,0.035714


In [40]:
pd.DataFrame(result['p-value_pcx'])

,p-value_pcx
NAME,
lypd8,0.012866
ly6k,0.030303
pate2,0.030303
lynx1,0.035714
